# Behavioral Cloning

In [1]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn.utils
import sklearn.model_selection
import cv2
%matplotlib inline
print('Tensorflow version {0}'.format(tf.__version__))
print('Keras version {0}'.format(keras.__version__))

Using TensorFlow backend.


Tensorflow version 1.3.0
Keras version 2.0.8


In [2]:
def change_brightness(img, lower_bound=0, upper_bound=255):
    '''
    Uniformly changes the brightness of an image randomly without overflowing.
    Change in brightness limited by original brightness of image
    '''
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    img[:,:,2] = img[:,:,2]/2 + np.random.randint(lower_bound, upper_bound)/2
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    return img

In [3]:
# def bin_angles(y, n_bins = 20):
#     '''Computes a bin number for each steering angle in order to balance classes'''
#     return np.floor((y - np.min(y))*n_bins/(np.max(y) - np.min(y)))

def bin_weights(y, n_bins=10):
    '''Balances numbered classes and returns array of weights to apply to each class'''
    y_bins = np.floor((y - np.min(y))*n_bins/(np.max(y) - np.min(y)))
    class_weight = sklearn.utils.class_weight.compute_class_weight('balanced', 
                                                                   np.unique(y_bins), 
                                                                   y_bins)
    class_dict = dict(zip(np.unique(y_bins), class_weight))
    return np.vectorize(class_dict.get)(y_bins)

## File I/O
Want to read in a directory of jpg files and write out hdf5

### Unbalanced Classes
We would like to have an equal distribution of steering angles to reduce the bias for zero driving angle. We will apply sample weights to balance the effect of each class.

In [4]:
# Separate CSV into numpy array of strings and numpy array of floats
file_paths = np.genfromtxt('./driving_data/driving_log.csv', dtype=np.str, usecols=(0,1,2), delimiter=',')
homogenous_samples = np.genfromtxt('./driving_data/driving_log.csv', delimiter=',', usecols=(3,4,5,6))
# Compute sample weights
steering_angles = homogenous_samples[:,0]
homogenous_samples = np.hstack((homogenous_samples, 
                        np.expand_dims(bin_weights(steering_angles), axis=1)))

In [ ]:
plt.hist(steering_angles, bins=10)

Save files to directory so that we can read them with `flow_from_directory`. Shoving them into an array causes memory errors.

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D
from keras.layers import MaxPooling2D, Cropping2D

model = Sequential()
model.add(Cropping2D(cropping=((20,8), (0,0)), input_shape=(64,128,3)))
model.add(Lambda(lambda x: x/255.0 - 0.5))
model.add(Conv2D(24, (5,5), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(36, (5,5), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(48, (5,5), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer=Adam(lr=0.0001))
history_object = model.fit(X, y, validation_split=0.2, 
          sample_weight=y_weights, shuffle=True, verbose=2)
model.save('model{0}.h5'.format(np.datetime_as_string(np.datetime64('now'))))

### L-R Camera compensation

A correction factor of 0.1 is added to the left camera and subtracted from the right camera to compensate for the lateral position of the camera relative to the center. The value of the correction factor should be evaluated.

In [65]:
def generator(file_paths, samples, batch_size, lr_shift):
    num_samples = len(samples)
    if len(samples) != len(file_paths):
        print('TODO: Error Message, incompatible arrays {0} {1}'.format(len(samples), len(file_paths)))
    while True:
        file_paths, samples = sklearn.utils.shuffle(file_paths, samples)
        for offset in range(0, num_samples, batch_size):
            batch_files = file_paths[offset:offset + batch_size]
            batch_samples = samples[offset:offset + batch_size]
            
            images = []
            measurements = []
            weights = []
            for batch_file in batch_files:
                #Read images from file
                center_file  = batch_file[0]
                left_file    = batch_file[1]
                right_file   = batch_file[2]
                center_image = mpimg.imread(center_file)
                left_image   = mpimg.imread(left_file)
                right_image  = mpimg.imread(right_file)
                #Change brightness of images
                center_image = change_brightness(center_image)
                left_image = change_brightness(left_image)
                right_image = change_brightness(right_image)
                images.append(center_image)
                images.append(left_image)
                images.append(right_image)
            
            for batch_sample in batch_samples:
                steer_angle  = batch_sample[0]
                sample_weight = batch_sample[4]
                
                #Steering angle offsets
                measurements.append(steer_angle)
                measurements.append(steer_angle+lr_shift)
                measurements.append(steer_angle-lr_shift)
                weights += [sample_weight]*3
                
            X_train = np.array(images)
            y_train = np.array(measurements)
            y_weights = np.array(weights)
            yield sklearn.utils.shuffle(X_train, y_train, y_weights)

In [66]:
BATCH_SIZE = 32
SHIFT = 0.1
train_files, val_files, train_samples, val_samples = \
    sklearn.model_selection.train_test_split(file_paths, homogenous_samples, test_size=0.2)
train_generator = generator(train_files, train_samples, batch_size=BATCH_SIZE, lr_shift=SHIFT)
validation_generator = generator(val_files, val_samples, batch_size=BATCH_SIZE, lr_shift=SHIFT)

In [67]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D
from keras.layers import MaxPooling2D, Cropping2D, Dropout

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: x/255.0 - 0.5))
model.add(Conv2D(36, (5,5), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(48, (3,3), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='elu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer=Adam(lr=0.0001))
history_object = model.fit_generator(train_generator, 
                                     steps_per_epoch=len(train_samples)/BATCH_SIZE,
                                     validation_data=validation_generator, 
                                     validation_steps=len(val_samples)/BATCH_SIZE,
                                     epochs=10, 
                                     verbose=2)
model.save('model{0}.h5'.format(np.datetime_as_string(np.datetime64('now'))))

Epoch 1/10
207s - loss: 0.1758 - val_loss: 0.1286
Epoch 2/10
203s - loss: 0.1046 - val_loss: 0.0887
Epoch 3/10
203s - loss: 0.0868 - val_loss: 0.0825
Epoch 4/10
206s - loss: 0.0756 - val_loss: 0.0792
Epoch 5/10
201s - loss: 0.0669 - val_loss: 0.0720
Epoch 6/10
202s - loss: 0.0580 - val_loss: 0.0704
Epoch 7/10
208s - loss: 0.0534 - val_loss: 0.0710
Epoch 8/10
208s - loss: 0.0483 - val_loss: 0.0701
Epoch 9/10
201s - loss: 0.0440 - val_loss: 0.0729
Epoch 10/10
210s - loss: 0.0408 - val_loss: 0.0661


In [68]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_10 (Cropping2D)   (None, 90, 320, 3)        0         
_________________________________________________________________
lambda_10 (Lambda)           (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 90, 320, 36)       2736      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 45, 160, 36)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 45, 160, 48)       15600     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 22, 80, 48)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 22, 80, 64)        27712     
__________

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
# center = mpimg.imread(log['CenterImg'][0])
# left = mpimg.imread(log['LeftImg'][0])
# right = mpimg.imread(log['RightImg'][0])
fig = plt.figure(figsize=(12,3))
plt.suptitle('Car Images')
axl = plt.subplot(1,3,1)
plt.imshow(left)
axc = plt.subplot(1,3,2)
plt.imshow(center)
axr = plt.subplot(1,3,3)
plt.imshow(right)
plt.show()